In [7]:
# import packages
# general tools
import numpy as np
# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

In [8]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """
    if x not in permitted_list:
        x = permitted_list[-1]
    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]
    return binary_encoding

In [9]:
def get_atom_features(atom,
                      use_chirality = True,
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """
    # define list of permitted atoms

    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']

    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms

    # compute atom features

    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)

    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])

    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])

    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])

    is_in_a_ring_enc = [int(atom.IsInRing())]

    is_aromatic_enc = [int(atom.GetIsAromatic())]

    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]

    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]

    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]
    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled

    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc

    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc
    return np.array(atom_feature_vector)

In [10]:
def get_bond_features(bond,
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """
    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)

    bond_is_conj_enc = [int(bond.GetIsConjugated())]

    bond_is_in_ring_enc = [int(bond.IsInRing())]

    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc

    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc
    return np.array(bond_feature_vector)

In [22]:
def create_minimal_hypergraph_with_labels(smiles):
    """
    Minimaler Hypergraph mit Beschriftungen + Node-/Edge-Features:
      - Hyperedges: Gesamtes Molekül, Ringe, Heteroatome, pos/neg Regionen, Rest
      - node_features: numpy-Array pro Knoten (Atome + Drug-Knoten)
      - edge_features: numpy-Array pro Hyperedge (Typ als One-Hot)
    """
    mol = Chem.MolFromSmiles(smiles)
    nums_atom = mol.GetNumAtoms()
    drug_node = nums_atom

    # 0) Hyperedges sammeln
    hyperedges = {}
    # 0a) Gesamtes Molekül
    hyperedges["drug"] = set(range(nums_atom)) | {drug_node}

    # 1) Ringe
    for i, ring in enumerate(mol.GetRingInfo().AtomRings()):
        hyperedges[f"ring_{i}"] = set(ring)

    # 2) Heteroatome
    hetero = {a.GetIdx() for a in mol.GetAtoms() if a.GetSymbol() in ('S','N','O')}
    if len(hetero) > 1:
        hyperedges["hetero"] = hetero

    # 3) Gasteiger Partial Charges
    AllChem.ComputeGasteigerCharges(mol)
    pos = {i for i in range(nums_atom)
           if float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge')) > 0}
    neg = {i for i in range(nums_atom)
           if float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge')) < 0}
    if len(pos) > 1: hyperedges["pos_charge"] = pos
    if len(neg) > 1: hyperedges["neg_charge"] = neg

    # 4) Rest
    covered = set().union(*hyperedges.values())
    rest = set(range(nums_atom)) - covered
    if len(rest) > 1:
        hyperedges["rest"] = rest

    # alle möglichen Hyperedge-Typen für One-Hot
    edge_types = ["drug", "ring", "hetero", "pos_charge", "neg_charge", "rest", "Unknown"]

    # 5) Node-Features
    node_features = {}
    # Atome
    for idx in range(nums_atom):
        atom = mol.GetAtomWithIdx(idx)
        node_features[idx] = get_atom_features(atom)
    # Drug-Knoten: Nullvektor derselben Länge
    feat_len = node_features[0].shape[0]
    node_features[drug_node] = np.zeros(feat_len, dtype=float)

    # 6) Edge-Features
    edge_features = {}
    for name in hyperedges:
        # Typ extrahieren: „ring_*“ → „ring“, sonst Name direkt
        etype = "ring" if name.startswith("ring_") else name
        # One-Hot
        edge_features[name] = np.array(one_hot_encoding(etype, edge_types), dtype=int)

    return hyperedges, node_features, edge_features

# Beispielaufruf
smiles = "CN1CCN(Cc2ccc(cc2)C(=O)Nc2ccc(C)c(Nc3nccc(n3)-c3cccnc3)c2)CC1"
H, n_feats, e_feats = create_minimal_hypergraph_with_labels(smiles)

In [23]:
H

{'drug': {0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37},
 'ring_0': {1, 2, 3, 4, 35, 36},
 'ring_1': {6, 7, 8, 9, 10, 11},
 'ring_2': {15, 16, 17, 18, 20, 34},
 'ring_3': {22, 23, 24, 25, 26, 27},
 'ring_4': {28, 29, 30, 31, 32, 33},
 'hetero': {1, 4, 13, 14, 21, 23, 27, 32},
 'pos_charge': {2, 3, 5, 9, 12, 15, 20, 22, 24, 26, 28, 31, 33, 35, 36},
 'neg_charge': {0,
  1,
  4,
  6,
  7,
  8,
  10,
  11,
  13,
  14,
  16,
  17,
  18,
  19,
  21,
  23,
  25,
  27,
  29,
  30,
  32,
  34}}

In [24]:
e_feats

{'drug': array([1, 0, 0, 0, 0, 0, 0]),
 'ring_0': array([0, 1, 0, 0, 0, 0, 0]),
 'ring_1': array([0, 1, 0, 0, 0, 0, 0]),
 'ring_2': array([0, 1, 0, 0, 0, 0, 0]),
 'ring_3': array([0, 1, 0, 0, 0, 0, 0]),
 'ring_4': array([0, 1, 0, 0, 0, 0, 0]),
 'hetero': array([0, 0, 1, 0, 0, 0, 0]),
 'pos_charge': array([0, 0, 0, 1, 0, 0, 0]),
 'neg_charge': array([0, 0, 0, 0, 1, 0, 0])}

In [25]:
n_feats

{0: array([1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01032802, 0.33333333, 0.15789474, 1.        ,
        0.        , 0.        , 0.        , 0

In [11]:
# convert SMILES to RDKit mol object
mol = Chem.MolFromSmiles("CN1CCN(Cc2ccc(cc2)C(=O)Nc2ccc(C)c(Nc3nccc(n3)-c3cccnc3)c2)CC1")
# get feature dimensions
n_nodes = mol.GetNumAtoms()
n_edges = 2*mol.GetNumBonds()
unrelated_smiles = "O=O"
unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

In [12]:
n_nodes

37

In [13]:
n_edges

82

In [15]:
get_atom_features(unrelated_mol.GetAtomWithIdx(0))

array([0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04468008, 0.08333333, 0.02631579, 1.        ,
       0.        , 0.        , 0.        , 1.        , 0.     

In [17]:
get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1))

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

def one_hot_encoding(x, permitted_list):
    enc = [int(x == p) for p in permitted_list]
    if sum(enc) == 0:
        enc[-1] = 1  # Unknown
    return enc

def get_atom_features(atom,
                      use_chirality=True,
                      hydrogens_implicit=True):
    # Basis-Features wie zuvor…
    permitted_atoms = ['C','N','O','S','F','Cl','Br','P','Si','Unknown']
    atom_type = one_hot_encoding(atom.GetSymbol(), permitted_atoms)
    degree = one_hot_encoding(atom.GetDegree(), [0,1,2,3,4,"MoreThanFour"])
    charge = one_hot_encoding(atom.GetFormalCharge(), [-3,-2,-1,0,1,2,3,"Extreme"])
    hybrid = one_hot_encoding(str(atom.GetHybridization()),
                              ["SP","SP2","SP3","OTHER"])
    in_ring = [int(atom.IsInRing())]
    aromatic = [int(atom.GetIsAromatic())]
    mass = [(atom.GetMass()-10.812)/116.092]
    # H-Bindungsspender/-akzeptor
    is_donor = int(atom.GetSymbol() in ("N","O") and atom.GetTotalNumHs()>0)
    is_acceptor = int(atom.GetSymbol() in ("N","O") and atom.GetTotalNumHs()<3)
    hb_donor = [is_donor]
    hb_acceptor = [is_acceptor]
    vec = (atom_type + degree + charge + hybrid +
           in_ring + aromatic + mass +
           hb_donor + hb_acceptor)
    if use_chirality:
        chir = one_hot_encoding(str(atom.GetChiralTag()),
                                ["CHI_UNSPECIFIED","CHI_TETRAHEDRAL_CW",
                                 "CHI_TETRAHEDRAL_CCW","CHI_OTHER"])
        vec += chir
    if hydrogens_implicit:
        hcount = one_hot_encoding(atom.GetTotalNumHs(), [0,1,2,3,4,"MoreThanFour"])
        vec += hcount
    return np.array(vec, dtype=float)

def get_bond_features(bond, use_stereo=True):
    types = [Chem.rdchem.BondType.SINGLE,
             Chem.rdchem.BondType.DOUBLE,
             Chem.rdchem.BondType.TRIPLE,
             Chem.rdchem.BondType.AROMATIC,
             "Unknown"]
    bt = one_hot_encoding(bond.GetBondType(), types)
    conj = [int(bond.GetIsConjugated())]
    ring = [int(bond.IsInRing())]
    vec = bt + conj + ring
    if use_stereo:
        st = one_hot_encoding(str(bond.GetStereo()),
                              ["STEREOZ","STEREOE","STEREOANY","STEREONONE"])
        vec += st
    return np.array(vec, dtype=float)

def create_enhanced_hypergraph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    n = mol.GetNumAtoms()
    drug_node = n
    hyperedges = {}

    # 1) Gesamtes Molekül
    hyperedges["drug"] = set(range(n)) | {drug_node}

    # 2) Ringe (aromatisch vs. aliphatisch)
    for i, ring in enumerate(mol.GetRingInfo().AtomRings()):
        atoms = set(ring)
        if all(mol.GetAtomWithIdx(j).GetIsAromatic() for j in ring):
            hyperedges[f"ring_aromatic_{i}"] = atoms
        else:
            hyperedges[f"ring_aliphatic_{i}"] = atoms

    # 3) Heteroatome
    hetero = {a.GetIdx() for a in mol.GetAtoms() if a.GetSymbol() in ("N","O","S")}
    if len(hetero)>1:
        hyperedges["hetero"] = hetero

    # 4) Partial Charges
    AllChem.ComputeGasteigerCharges(mol)
    pos = {i for i in range(n)
           if float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge'))>0}
    neg = {i for i in range(n)
           if float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge'))<0}
    if len(pos)>1: hyperedges["pos_charge"] = pos
    if len(neg)>1: hyperedges["neg_charge"] = neg

    # 5) H-Binderspender/-akzeptoren als Hyperedges
    donors   = {i for i in range(n)
                if get_atom_features(mol.GetAtomWithIdx(i))[ - (2 + (1 if True else 0)) ] == 1}
    acceptors= {i for i in range(n)
                if get_atom_features(mol.GetAtomWithIdx(i))[ - (1 + (1 if True else 0)) ] == 1}
    if len(donors)>1:   hyperedges["hbond_donor"] = donors
    if len(acceptors)>1:hyperedges["hbond_acceptor"] = acceptors

    # 7) Rest-Knoten
    covered = set().union(*hyperedges.values())
    rest = set(range(n)) - covered
    if len(rest)>1:
        hyperedges["rest"] = rest

    # Node-Features
    node_feats = {}
    for i in range(n):
        node_feats[i] = get_atom_features(mol.GetAtomWithIdx(i))
    # Drug-Knoten: Nullvektor
    dim = node_feats[0].shape[0]
    node_feats[drug_node] = np.zeros(dim, dtype=float)

    # Edge-Features: Typ-One-Hot
    types = ["drug",
             "ring_aromatic","ring_aliphatic",
             "hetero","pos_charge","neg_charge",
             "hbond_donor","hbond_acceptor",
             *FUNCTIONAL_GROUP_SMARTS.keys(),
             "rest","Unknown"]
    edge_feats = {}
    for key, nodes in hyperedges.items():
        # Basis-Typ extrahieren
        base = key.split("_")[0] if key.startswith("ring_") or key in FUNCTIONAL_GROUP_SMARTS else key
        edge_feats[key] = np.array(one_hot_encoding(base, types), dtype=int)

    return hyperedges, node_feats, edge_feats

# Beispiel
smiles = "CC(=O)NC1=CC=CC=C1O"
H, NF, EF = create_enhanced_hypergraph(smiles)
print("Hyperedges:", list(H.keys()))
print("Beispiel Node-Feature[0]:", NF[0])

Hyperedges: ['drug', 'ring_aromatic_0', 'hetero', 'pos_charge', 'neg_charge']
Beispiel Node-Feature[0]: [1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         0.         0.
 0.         1.         0.         0.         0.         0.
 0.         0.         1.         0.         0.         0.
 0.01032802 0.         0.         1.         0.         0.
 0.         0.         0.         0.         1.         0.
 0.        ]
